In [8]:
# Standard imports wo work with data, graphs, APIs, etc.

# Compatibility with Python 3
from __future__ import print_function, division

import pandas as pd
import geopandas as gpd
import numpy as np
import json
import requests
import urllib2
import urllib
import ast
from cStringIO import StringIO
from pandas.tools.plotting import scatter_matrix
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
from scipy.interpolate import *
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
import zipfile
from geopandas import GeoDataFrame
import pysal as ps
from fiona.crs import from_epsg
import matplotlib.colors as mcolors
import operator
from matplotlib.colors import ListedColormap    
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pylab as plt
import seaborn as sns
import csv
import dateutil
import os
import psycopg2

%matplotlib inline

In [9]:
## read json file for password 
import json
json_data = open("password.json").read() #reading password
passw = json.loads(json_data)

PASSWORD = str(passw["PASSWORD"])

In [10]:
# https://support.dominodatalab.com/hc/en-us/articles/207346026-Connecting-to-Redshift
# OAG Redshift Reference:
# https://docs.google.com/document/d/1FsTqbR9tk0Z7vxbDJ1vaNiEHfKEinxtOTU8YvBnUNI8/edit

HOST = "research1.cwcou8cgfav2.us-east-1.redshift.amazonaws.com"
PORT = 5439 # redshift default
USER = "cusp2017"

# Password either from environmental variable or input raw
# PASSWORD = os.environ['REDSHIFT_PASSWD']
#PASSWORD = raw_input()

In [11]:
# oag_civis
DATABASE = 'oag_civis'

def db_connection():
    conn = psycopg2.connect(
        host=HOST,
        port=PORT,
        user=USER,
        password=PASSWORD,
        database=DATABASE,
    )
    return conn


##### DOF datasets exploration

In [12]:
example_query = "Select * from agn_buildings.acris_codes;"
# select top 100 * from [buildings_and_violations].[dbo].[OATH_ECB_Violations]

conn = db_connection()
try:
    df = pd.read_sql(example_query, conn)
finally:
    conn.close()
df.head()

,record_type,doc_type,doc_type_description,class_code_description,party1_type,party2_type,party3_type
0,D,AGMT,AGREEMENT,MORTGAGES & INSTRUMENTS,PARTY 1,PARTY 2,None
1,D,ASPM,ASSUMPTION OF MORTGAGE,MORTGAGES & INSTRUMENTS,MORTGAGER/BORROWER,MORTGAGEE/LENDER,None
2,D,ASTU,UNIT ASSIGNMENT,DEEDS AND OTHER CONVEYANCES,GRANTOR/SELLER,GRANTEE/BUYER,None
3,D,BOND,BOND,OTHER DOCUMENTS,DEBTOR,SECURED PARTY,None
4,D,CERR,CERTIFICATE OF REDUCTION,MORTGAGES & INSTRUMENTS,MORTGAGER/BORROWER,MORTGAGEE/LENDER,None


In [31]:
example_query = "Select * from agn_buildings.acris_master as M where M.doc_type = 'DEED' or \
M.doc_type = 'MTGE';"

conn = db_connection()
try:
    df = pd.read_sql(example_query, conn)
finally:
    conn.close()
df.head()

,document_id,record_type,crfn,borough,doc_type,doc_amount,collateral,slid,rptt,reel_year,reel_nbr,reel_page,file_number,_transferred,source,good_through_date,assessment_date,modified_date,doc_date,recorded_filed
0,2004072702987008,A,2005000052014,1,DEED,25900.0,None,None,0,0,0,0,0,100.0,RP,2015-07-31,None,2005-01-26,0204-01-01,2005-01-26
1,2010012601069001,A,2010000033842,1,DEED,0.0,None,None,0,0,0,0,0,100.0,RP,2015-07-31,None,2010-01-29,1020-01-01,2010-01-29
2,BK_6730035200034,A,None,3,DEED,0.0,None,None,0,1967,352,34,0,0.0,RP,2015-07-31,None,1967-01-01,1967-01-01,1967-01-01
3,BK_7040040301728,A,None,4,DEED,0.0,None,None,0,1970,403,1728,0,0.0,RP,2015-07-31,None,1970-01-01,1970-01-01,1970-01-01
4,FT_3840004438284,A,None,3,DEED,0.0,None,None,0,1994,3378,690,0,0.0,RP,2015-07-31,None,1994-09-29,1970-01-01,1994-09-14


In [18]:
df.shape

(17862947, 20)

In [25]:
example_query = "Select * from agn_buildings.acris_parties;"


conn = db_connection()
try:
    df = pd.read_sql(example_query, conn)
finally:
    conn.close()
df.head()

,document_id,record_type,party_type,name,address_1,address_2,country,city,state,zip,source,good_through_date
0,2004012900738002,P,1,"JFK PLAZA, LLC",146-27/35 167TH STREET,None,US,JAMAICA,NY,11434,RP,2015-07-31
1,BK_7120015501765,P,1,ALUMA LIFE PRODUCTS INC,None,None,None,None,None,None,RP,2015-07-31
2,FT_2000000797100,P,1,"HERON, DAVID",3383 WALLACE AVE,None,US,BX,NY,10467,RP,2015-07-31
3,FT_4440005397544,P,1,"HOLUKA, ANNA",76-65 172ND ST,None,US,FLUSHING,NY,11366,RP,2015-07-31
4,FT_2430006678043,P,1,"BERNARD, RUDOLPH",None,None,None,None,None,None,RP,2015-07-31


In [26]:
example_query = "Select * from agn_buildings.acris_legal;"


conn = db_connection()
try:
    df = pd.read_sql(example_query, conn)
finally:
    conn.close()
df.head()

,document_id,record_type,borough,block,lot,easement,partial_lot,air_rights,subterranean_rights,property_type,street_number,street_name,unit,source,bbl,good_through_date
0,2015072300565005,L,1,1468,46,N,E,N,N,AP,1382,FIRST AVENUE,None,RP,1-1468-46,2015-08-31
1,2015080400007002,L,1,1863,52,N,E,N,N,AP,132,WEST 109 STREET,None,RP,1-1863-52,2015-08-31
2,2015073100523003,L,1,1323,122,N,E,N,N,AP,253,EAST 49TH STREET,None,RP,1-1323-122,2015-08-31
3,2006060100245001,L,1,2174,192,N,E,N,N,AP,31,ARDEN STREET,None,RP,1-2174-192,2015-07-31
4,2015081200977001,L,1,901,5,N,E,N,N,AP,253,THIRD AVENUE,None,RP,1-901-5,2015-08-31


In [47]:
example_query = "Select parties.DOCUMENT_ID, parties.party_type, parties.record_type, \
parties.name, parties.address_1, parties.address_2, \
M.doc_type, M.doc_amount, M.doc_date \
from agn_buildings.acris_parties as parties \
left join agn_buildings.acris_master as M on parties.DOCUMENT_ID = M.document_id \
where parties.name ilike '%ciampa%' AND (M.doc_type = 'DEED' OR M.doc_type = 'MTGE');"


conn = db_connection()
try:
    df = pd.read_sql(example_query, conn)
finally:
    conn.close()
df.head()

,document_id,party_type,record_type,name,address_1,address_2,doc_type,doc_amount,doc_date
0,2007102900842001,2,P,CIAMPA NORTH LLC,136-26 37TH AVENUE,None,DEED,0.0,2002-01-01
1,2007102900842001,1,P,"CIAMPA, DOMINICK",136-26 37TH AVENUE,None,DEED,0.0,2002-01-01
2,2007102900842001,1,P,"CIAMPA, BENJAMIN",136-26 37TH AVENUE,None,DEED,0.0,2002-01-01
3,2007102900842001,1,P,"CIAMPA, JOSEPH D JR",136-26 37TH AVENUE,None,DEED,0.0,2002-01-01
4,2007102900842001,1,P,"CIAMPA, DOUGLAS",136-26 37TH AVENUE,None,DEED,0.0,2002-01-01


In [44]:
df.shape

(1572, 6)

In [10]:
# agn_buildings - 311 complaints

sam_query = "select * from agn_buildings.complaints_311 limit 10;"

conn =db_connection()
try:
    df = pd.read_sql(sam_query, conn)
finally:
    conn.close()
    
df.head(3)

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,bridge_highway_direction,road_ramp,bridge_highway_segment,garage_lot_name,ferry_direction,ferry_terminal_name,latitude,longitude,location,bbl
0,16819147,2010-06-07 11:52:24,2010-06-14 00:00:00,DOB,Department of Buildings,General Construction/Plumbing,Building Permit - None,None,10007,20 WARREN STREET,...,None,None,None,None,None,None,40.7140300886328,-74.00763646599842,"(40.7140300886328, -74.00763646599842)",1-135-9
1,16819151,2010-06-07 12:46:52,2014-01-25 00:00:00,DOB,Department of Buildings,Special Projects Inspection Team (SPIT),Illegal Hotel Rooms In Residential Building,None,10016,3 PARK AVENUE,...,None,None,None,None,None,None,40.74678477229352,-73.98154726128544,"(40.74678477229352, -73.98154726128544)",1-889-7501
2,16819156,2010-06-07 10:16:00,2010-06-09 12:00:00,DSNY,BCC - Brooklyn South,Sanitation Condition,15 Street Cond/Dump-Out/Drop-Off,Street,11204,346 AVENUE P,...,None,None,None,None,None,None,40.60860337537493,-73.97484702153419,"(40.60860337537493, -73.97484702153419)",3-6631-1


In [21]:
## agn_buildings - dob_complaints, dob_violations, dob_permits, hpd_complaints, hpd_violations, hpd_litigation

sample_query = "select top 100 * from agn_buildings.acris_rp_master;"

conn = db_connection()
try:
    df = pd.read_sql(sample_query, conn)
finally:
    conn.close()
    
df.head()

,document_id,record_type,crfn,borough,doc_type,doc_date,doc_amount,recorded_filed,modified_date,reel_year,reel_nbr,reel_page,_transferred,good_through_date
0,2011021501154001,A,2011000063238,2,AALR,01/01/2009,0.0,02/22/2011,02/22/2011,0,0,0,0.0,07/31/2015
1,2011012100493002,A,2011000030211,1,AALR,01/01/2010,0.0,01/25/2011,01/25/2011,0,0,0,0.0,07/31/2015
2,2011012100636002,A,2011000030492,1,AALR,01/01/2010,0.0,01/25/2011,01/25/2011,0,0,0,0.0,07/31/2015
3,2011012100782003,A,2011000030264,1,AALR,01/01/2010,0.0,01/25/2011,01/25/2011,0,0,0,0.0,07/31/2015
4,2011012100493006,A,2011000030215,1,AALR,01/01/2010,0.0,01/25/2011,01/25/2011,0,0,0,0.0,07/31/2015


In [22]:
## agn_buildings -dob_violations

sample_query = "select * from agn_buildings.dob_violations limit 10;"

conn = db_connection()
try:
    df = pd.read_sql(sample_query, conn)
finally:
    conn.close()
    
df.head()

,isn_dob_bis_viol,boro,bin,block,lot,issue_date,violation_type_code,violation_number,house_number,street,disposition_date,disposition_comments,device_number,description,ecb_number,number,violation_category,violation_type,issue_date_date
0,1853880,4,4314540.0,01863,07502,20140515,LBLVIO,19551,48-21,92 STREET,20160516,"CIAKMH DISMISSED, INV #61675679",00067775,VIOLATION ISSUED FOR FAILURE TO FILE ANNUAL BO...,,V*051514LBLVIO19551,V*-DOB VIOLATION - DISMISSED,LBLVIO-LOW PRESSURE BOILER,2014-05-15
1,142871,3,3117400.0,05111,00024,19920407,LL1081,02749,2820,SNYDER AVENUE,20160516,JMA CAT 5 PERFORMED 5-14-15,E9202749,1,,V*040792LL108102749,V*-DOB VIOLATION - DISMISSED,LL1081-LOCAL LAW 10/81 - ELEVATOR,1992-04-07
2,348436,4,4054168.0,02344,00030,19950417,LL1081,03715,59-06,LAUREL HILL BOULEVARD,20160516,JMA 2015 SAT CAT 1 PERFORMED 3-4-15,E9503715,,,V*041795LL108103715,V*-DOB VIOLATION - DISMISSED,LL1081-LOCAL LAW 10/81 - ELEVATOR,1995-04-17
3,193378,4,,15544,00028,19930628,LL6291,14383,1431,CENTRAL AVENUE,,,00084457,,,V062893LL629114383,V-DOB VIOLATION - ACTIVE,LL6291-LOCAL LAW 62/91 - BOILERS,1993-06-28
4,1910708,4,4436638.0,00098,00009,20141031,EVCAT1,02893,47-07,PEARSON PLACE,20160516,JMA CAT 5 PERFORMED 5-21-15,4F173,VIOLATION ISSUED TO ELEVATOR-FAILURE TO FILE C...,,V*103114EVCAT102893,V*-DOB VIOLATION - DISMISSED,EVCAT1-ELEVATOR ANNUAL INSPECTION / TEST,2014-10-31


In [24]:
## agn_buildings -hpd complaints

sample_query = "select * from agn_buildings.hpd_complaints limit 10;"

conn = db_connection()
try:
    df = pd.read_sql(sample_query, conn)
finally:
    conn.close()
    
df.head(3)

,complaintid,buildingid,boroughid,borough,housenumber,streetname,zip,block,lot,apartment,communityboard,recieveddate,referencenumber,statusid,status,statusdate,source,last_mod_date,bbl
0,6446422,545,1,MANHATTAN,261,1 AVENUE,10003.0,922,20,,6,6/9/2013,1-1-860192811,None,,,Complaint,20130630,1-922-20
1,6443149,5999,1,MANHATTAN,641,AMSTERDAM AVENUE,10025.0,1222,1,7F,7,6/4/2013,,None,,,Complaint,20130630,1-1222-1
2,6421669,13279,1,MANHATTAN,58,EAST 34 STREET,10016.0,863,51,6,5,5/7/2013,,None,,,Complaint,20130630,1-863-51


In [43]:
# dob_ecbviolations

sample_query = "select * from agn_buildings.dob_ecbviolations limit 10;"

conn = db_connection()
try:
    df = pd.read_sql(sample_query, conn)
finally:
    conn.close()
    
df.head(3)

,isn_dob_bis_extract,ecb_violation_number,ecb_violation_status,dob_violation_number,bin,boro,block,lot,hearing_date,hearing_time,...,infraction_code8,section_law_description8,infraction_code9,section_law_description9,infraction_code10,section_law_description10,aggravated_level,hearing_status,certification_status,hearing_date_date
0,758705,34830294Z,RESOLVE,012910STFJH04,4080791,4,03388,0022,20100525,1030,...,,,,,,,NO,DEFAULT,CERTIFICATE ACCEPTED,2010-05-25
1,453910,38203404N,RESOLVE,052609E2178B01,2003818,2,02568,0057,20090724,1030,...,,,,,,,NO,IN VIOLATION,CERTIFICATE ACCEPTED,2009-07-24
2,257974,38087901Z,RESOLVE,062599E5Y825A6,1073089,1,01517,07501,19991007,830,...,,,,,,,,IN VIOLATION,CERTIFICATE ACCEPTED,1999-10-07


In [44]:
df.columns

Index([u'isn_dob_bis_extract', u'ecb_violation_number',
       u'ecb_violation_status', u'dob_violation_number', u'bin', u'boro',
       u'block', u'lot', u'hearing_date', u'hearing_time', u'served_date',
       u'issue_date', u'severity', u'violation_type', u'respondent_name',
       u'respondent_house_number', u'respondent_street', u'respondent_city',
       u'respondent_zip', u'violation_description', u'penality_imposed',
       u'amount_paid', u'balance_due', u'infraction_code1',
       u'section_law_description1', u'infraction_code2',
       u'section_law_description2', u'infraction_code3',
       u'section_law_description3', u'infraction_code4',
       u'section_law_description4', u'infraction_code5',
       u'section_law_description5', u'infraction_code6',
       u'section_law_description6', u'infraction_code7',
       u'section_law_description7', u'infraction_code8',
       u'section_law_description8', u'infraction_code9',
       u'section_law_description9', u'infraction_code10

In [36]:
## agn_buildings -taxbillsnyc_joined

sample_query = "select * from agn_buildings.taxbillsnyc_joined limit 10;"

conn = db_connection()
try:
    df = pd.read_sql(sample_query, conn)
finally:
    conn.close()
    
df.head(3)

,borough,ucbbl,_2007uc,_2007est,_2007dhcr,_2007abat,_2008uc,_2008est,_2008dhcr,_2008abat,...,ownername,numbldgs,numfloors,unitsres,unitstotal,yearbuilt,condono,lon,lat,bbl
0,MN,1000077501,21.0,N,None,None,17.0,N,None,None,...,COENTIES SLIP APT CO,1,6.0,42,43,1920,735,-74.0108,40.7034,1-7-7501
1,MN,1000160015,208.0,N,None,None,208.0,N,None,None,...,DEMATTEIS BATTERY PAR,1,9.0,208,209,1998,0,-74.0182,40.7067,1-16-15
2,MN,1000160180,293.0,N,None,None,293.0,N,None,None,...,None,1,27.0,293,294,2002,0,-74.0161,40.7166,1-16-180


In [49]:
df.columns

Index([u'loboro', u'loblock', u'lolot', u'lobblscc', u'hiboro', u'hiblock',
       u'hilot', u'hibblscc', u'boro', u'block', u'lot', u'bblscc',
       u'billboro', u'billblock', u'billlot', u'billbblscc', u'condoflag',
       u'condonum', u'coopnum', u'numbf', u'numaddr', u'vacant', u'interior'],
      dtype='object')

In [5]:
# hpd registration contact

sample_query = "select * from agn_buildings.hpd_registration_contact where firstname ilike '%CIAMPA%' or  lastname ilike '%CIAMPA%';"
conn = db_connection()
try:
    df = pd.read_sql(sample_query, conn)
finally:
    conn.close()
    
df.head(3)

,registrationcontactid,registrationid,type,contactdescription,corporationname,title,firstname,middleinitial,lastname,businesshousenumber,businessstreetname,businessapartment,businesscity,businessstate,businesszip,source,last_mod_date
0,42043004,420430,Agent,GEN.PART,CIAMPA MANAGEMENT CORP,,DOUGLAS,,CIAMPA,136-26,37 AVENUE,,FLUSHING,NY,11354,RegistrationContact,20130331
1,42043006,420430,Officer,GEN.PART,,PARTNER,JOSEPH,,CIAMPA,136-26,37 AVENUE,,FLUSHING,NY,11354,RegistrationContact,20130331
2,42096905,420969,HeadOfficer,GEN.PART,,PARTNER,DOUGLAS,,CIAMPA,136-26,37 AVENUE,,FLUSHING,NY,11354,RegistrationContact,20130331


In [11]:
# hpd registration 

sample_query = "select * from agn_buildings.hpd_registration limit 10;"
conn = db_connection()
try:
    df = pd.read_sql(sample_query, conn)
finally:
    conn.close()
    
df.head(3)

,registrationid,buildingid,boroid,boro,housenumber,lowhousenumber,highhousenumber,streetname,streetcode,zip,block,lot,bin,communityboard,lastregistrationdate,registrationenddate,source,last_mod_date,bbl
0,10043,137806,5,STATEN ISLAND,716,716,716,HARRIS AVENUE,3181001010,10314,781,28,5020043.0,502,11/9/2010,5/1/2011,Registration,20130331,5-781-28
1,10060,138734,5,STATEN ISLAND,356,356,356,CARY AVENUE,2165001010,10310,165,24,5112956.0,501,11/8/2012,9/1/2013,Registration,20130331,5-165-24
2,10062,138748,5,STATEN ISLAND,41,41,41,HILLBROOK COURT,7041801010,10305,3005,7501,5116698.0,501,10/22/1999,9/30/2000,Registration,20130331,5-3005-7501
